In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn import preprocessing
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import train_test_split

# Data 위치
data_path = './data'


class CFG:
    SEED = 42


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


seed_everything(CFG.SEED)  # Seed 고정

train = pd.read_csv(data_path+'/train.csv')
test = pd.read_csv(data_path+'/test.csv')


def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x


train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)

train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)

for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

train_x = train_x.drop(columns=['father', 'mother', 'gender'])
test_x = test_x.drop(columns=['father', 'mother', 'gender'])

train_x = train_x.drop(columns=['SNP_03'])
test_x = test_x.drop(columns=['SNP_03'])

train_data_x, valid_data_x, train_data_y, valid_data_y = train_test_split(
    train_x, train_y, test_size=0.1, random_state=CFG.SEED)

cat_peature = ['SNP_10']

dtrain = Pool(train_data_x, label=train_data_y, cat_features=cat_peature)
dvalid = Pool(valid_data_x, label=valid_data_y, cat_features=cat_peature)

params = {'learning_rate': 0.8, 'depth': 10,
          'boosting_type': 'Plain',
          'iterations': 5000,
          'task_type': 'GPU',
          'max_bin': 20000
          }

model = CatBoostClassifier(**params)
model.fit(dtrain, eval_set=dvalid, early_stopping_rounds=1000, use_best_model=True)

preds = model.predict(test_x)
print('Done.')

submit = pd.read_csv(data_path+'/sample_submission.csv')

submit['class'] = class_le.inverse_transform(preds)

submit.to_csv('./submit.csv', index=False)
print('Write result.')
